# Fileupload

In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import plotly.express as px
from numpy import random

print(f"Running on PyMC v{pm.__version__}")

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Running on PyMC v4.2.2


In [2]:
import stan
import nest_asyncio
nest_asyncio.apply()


In [3]:
data = pd.read_csv("./logremoval/testdata.csv")

In [4]:
data["censored_in"] = 0

In [6]:
inflow = data[["inflow", "censored_in"]].dropna().astype('int32')

In [7]:
inflow

,inflow,censored_in
0,1091,0
1,1017,0
2,983,0
3,1015,0
4,1126,0
5,1041,0
6,1165,0
7,1042,0
8,1012,0
9,956,0


In [8]:
standata = {"Y": inflow["inflow"].values.tolist(),
           "N": len(data["inflow"].dropna().values),
            "cens":inflow["censored_in"].values.tolist(),
           "N_new":1}

In [9]:
with open('./logremoval/stanmodels/negbin.txt') as f:
    stanmodel = f.read()

In [10]:
posterior = stan.build(stanmodel, data=standata)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_3zutu2c0/model_mmkz5wx5.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_3zutu2c0/model_mmkz5wx5.stan', line 6, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [11]:
fit = posterior.sample(num_chains=4, num_samples=2000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:  25% (3000/12000)
Sampling:  50% (6000/12000)
Sampling:  75% (9000/12000)
Sampling: 100% (12000/12000)
Sampling: 100% (12000/12000), done.
Messages received during sampling:
  Gradient evaluation took 0.001201 seconds
  1000 transitions using 10 leapfrog steps per transition would take 12.01 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_07anxovq/model_mmkz5wx5.stan', line 28, column 8 to column 59)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the followin

In [12]:
np.quantile(df["y_sim.1"], q = [.1, .5, .9])

array([ 942., 1047., 1162.])

In [104]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,Intercept,shape,b_Intercept,y_sim.1
draws,,,,,,,,,,,
0,-89.318899,0.944855,0.778190,1.0,1.0,0.0,89.404156,7.015690,156.346678,7.015690,1109.0
1,-89.143434,0.983790,0.661152,2.0,7.0,0.0,89.856751,7.001553,78.557280,7.001553,1169.0
2,-88.571846,0.902217,0.706310,2.0,3.0,0.0,89.586103,7.044406,92.238049,7.044406,1307.0
3,-89.779113,0.995832,0.996970,2.0,3.0,0.0,90.305732,7.082151,85.202219,7.082151,1034.0
4,-88.860311,0.941678,0.778190,2.0,3.0,0.0,90.734621,7.053299,77.942783,7.053299,1329.0
...,...,...,...,...,...,...,...,...,...,...,...
7995,-89.085710,0.407810,0.996970,1.0,1.0,0.0,90.760821,7.043597,154.039549,7.043597,1203.0
7996,-88.749590,0.944952,0.778190,2.0,3.0,0.0,89.042561,7.043887,131.765336,7.043887,1043.0
7997,-97.063516,0.984971,0.661152,3.0,7.0,0.0,97.270296,6.906671,13.679165,6.906671,1090.0


In [31]:
import stan

schools_code = """
data {
  int<lower=1> N;  // total number of observations
  int Y[N];  // response variable
  int N_new;
 
}
transformed data {
}
parameters {
  real Intercept;  // temporary intercept for centered predictors
  real<lower=0> shape;  // shape parameter
}
transformed parameters {
}
model {
  // likelihood including constants
 
  // initialize linear predictor term
  vector[N] mu = Intercept + rep_vector(0.0, N);
  target += neg_binomial_2_log_lpmf(Y | mu, shape);

  // priors including constants
  target += student_t_lpdf(Intercept | 3, 11.2, 3);
  target += gamma_lpdf(shape | 0.01, 0.01);
}
generated quantities {
  // actual population-level intercept
  real b_Intercept = Intercept;
  vector[N_new] y_sim;
  
  for (n in 1:N_new) {
    y_sim[n] = neg_binomial_2_rng(exp(b_Intercept), shape);
  }
  
}

"""




In [19]:
posterior = stan.build(stanmodel, data=standata)

Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.9s
Building: 7.0s
Building: 7.1s
Building: 

In file included from /home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing.hpp:5,
                 from /home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/model_header.hpp:17,
                 from /home/wseis/.cache/httpstan/4.8.2/models/hxpsbqyo/model_hxpsbqyo.cpp:2:
/home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp: In function 'void stan::model::assign(Mat1&&, const Mat2&, const char*, const stan::model::index_multi&, const stan::model::index_multi&)':
/home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp:401:9: warning: typedef 'using pair_type = struct std::pair<int, std::vector<int, stan::math::arena_allocator<int> > >' locally defined but not used [-Wunused-local-typedefs]
  401 |   using pair_type = std::pair<int, arena_vec>;
      |         ^~~~~~~~~
/home/wseis/.c

Building: 12.9s
Building: 13.0s
Building: 13.1s
Building: 13.2s
Building: 13.3s
Building: 13.4s


/home/wseis/.cache/httpstan/4.8.2/models/hxpsbqyo/model_hxpsbqyo.cpp: In instantiation of 'void model_hxpsbqyo_namespace::model_hxpsbqyo::transform_inits_impl(VecVar&, VecI&, VecVar&, std::ostream*) const [with VecVar = std::vector<double, std::allocator<double> >; VecI = std::vector<int>; stan::require_vector_t<T_y>* <anonymous> = 0; stan::require_vector_like_vt<std::is_integral, VecI>* <anonymous> = 0; std::ostream = std::basic_ostream<char>]':
/home/wseis/.cache/httpstan/4.8.2/models/hxpsbqyo/model_hxpsbqyo.cpp:434:69:   required from here
/home/wseis/.cache/httpstan/4.8.2/models/hxpsbqyo/model_hxpsbqyo.cpp:273:11: warning: variable 'pos__' set but not used [-Wunused-but-set-variable]
  273 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~


Building: 13.5s
Building: 13.6s
Building: 13.7s
Building: 13.8s
Building: 13.9s
Building: 14.0s
Building: 14.1s
Building: 14.2s
Building: 14.3s
Building: 14.4s
Building: 14.5s
Building: 14.6s
Building: 14.8s
Building: 14.9s
Building: 15.0s
Building: 15.1s
Building: 15.2s
Building: 15.3s
Building: 15.4s
Building: 15.5s
Building: 15.6s
Building: 15.7s
Building: 15.8s
Building: 15.9s
Building: 16.0s
Building: 16.1s
Building: 16.2s
Building: 16.3s
Building: 16.4s
Building: 16.5s
Building: 16.6s
Building: 16.8s
Building: 16.9s
Building: 17.0s
Building: 17.1s
Building: 17.2s
Building: 17.3s
Building: 17.4s
Building: 17.5s
Building: 17.6s
Building: 17.7s
Building: 17.8s
Building: 17.9s
Building: 18.0s
Building: 18.1s
Building: 18.2s
Building: 18.3s
Building: 18.4s
Building: 18.5s
Building: 18.6s
Building: 18.8s
Building: 18.9s
Building: 19.0s
Building: 19.1s
Building: 19.2s
Building: 19.3s
Building: 19.4s
Building: 19.5s
Building: 19.6s
Building: 19.7s
Building: 19.8s
Building: 19.9s
Building

Building: 25.0s, done.
Messages from stanc:
Warning in '/tmp/httpstan_vp2rjn6c/model_hxpsbqyo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_vp2rjn6c/model_hxpsbqyo.stan', line 6, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [20]:
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.6 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_m7vgbn66/model_hxpsbqyo.stan', line 26, column 8 to column 59)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  E

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_m7vgbn66/model_hxpsbqyo.stan', line 26, column 8 to column 59)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_m7vgb

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_m7vgbn66/model_hxpsbqyo.stan', line 26, column 8 to column 59)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter is inf, but must be positive finite! (in '/tmp/httpstan_m7vgb

In [22]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,Intercept,shape,b_Intercept
draws,,,,,,,,,,
0,-176.405334,0.974939,0.753966,2.0,3.0,0.0,176.582845,12.433205,0.384585,12.433205
1,-176.470426,1.000000,0.670341,2.0,3.0,0.0,177.994513,13.001632,0.270428,13.001632
2,-176.220810,0.977760,0.655481,3.0,7.0,0.0,177.466285,13.516837,0.495401,13.516837
3,-175.475333,1.000000,0.660635,2.0,3.0,0.0,176.450060,13.084177,0.432504,13.084177
4,-176.133773,0.981266,0.753966,3.0,7.0,0.0,177.036776,12.600805,0.529827,12.600805
...,...,...,...,...,...,...,...,...,...,...
3995,-176.515195,0.974251,0.660635,3.0,7.0,0.0,177.944063,12.550970,0.294984,12.550970
3996,-176.796419,0.853686,0.753966,3.0,7.0,0.0,176.941687,13.775223,0.372002,13.775223
3997,-175.744630,0.996769,0.670341,2.0,3.0,0.0,176.295137,12.986948,0.580321,12.986948


In [21]:
np.quantile(df["y_sim.1"], q = [.1, .5, .9])

KeyError: 'y_sim.1'

In [36]:
df.describe()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,Intercept,shape,b_Intercept,y_sim.1
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03
mean,-172.889600,0.929776,0.732964,2.136750,4.486000,0.0,173.878069,13.091787,0.428065,13.091787,5.530493e+05
std,1.095796,0.109142,0.118763,0.625817,2.100216,0.0,1.473432,0.487449,0.145428,0.487449,1.066072e+06
min,-180.752833,0.078163,0.589163,1.000000,1.000000,0.0,171.858928,11.326490,0.110083,11.326490,0.000000e+00
25%,-173.291019,0.912559,0.669892,2.000000,3.000000,0.0,172.783461,12.754488,0.324865,12.754488,2.741050e+04
50%,-172.544569,0.968918,0.712145,2.000000,3.000000,0.0,173.562825,13.051969,0.410556,13.051969,1.789560e+05
75%,-172.125946,0.995400,0.775218,3.000000,7.000000,0.0,174.609648,13.359121,0.515567,13.359121,6.413532e+05
max,-171.830732,1.000000,0.918404,3.000000,7.000000,0.0,181.792180,16.015817,1.200678,16.015817,2.125901e+07
